In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import Dataset,DataLoader
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

In [ ]:
torch.manual_seed(42)

In [ ]:

df=pd.read_csv('/content/fmnist_small.csv')

In [ ]:
df.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,9,0,0,0,0,0,0,0,0,0,...,0,7,0,50,205,196,213,165,0,0
1,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,1,0,0,0,...,142,142,142,21,0,3,0,0,0,0
3,8,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,8,0,0,0,0,0,0,0,0,0,...,213,203,174,151,188,10,0,0,0,0


In [ ]:
X=df.iloc[:,1:].values
y=df.iloc[:,0].values

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [ ]:
X_train=X_train/255.0
X_test=X_test/255.0

In [ ]:
#create customedatset class
class CustomDataset(Dataset):
  def __init__(self,features,labels):
    self.features=torch.tensor(features,dtype=torch.float32)
    self.labels=torch.tensor(labels,dtype=torch.long)

  def __len__(self):
    return len(self.features)

  def __getitem__(self,idx):
    return self.features[idx],self.labels[idx]

In [ ]:
#create train dataset object
train_dataset=CustomDataset(X_train,y_train)

In [ ]:
#create dataset object
test_dataset=CustomDataset(X_test,y_test)

In [ ]:
#create train and test loader
train_loader=DataLoader(train_dataset,batch_size=32,shuffle=True)
test_loader=DataLoader(test_dataset,batch_size=32,shuffle=False)

In [ ]:
#define NN class
class MyNN(nn.Module):
  def __init__(self,num_features):
    super().__init__()
    self.model=nn.Sequential(
        nn.Linear(num_features,128),
        nn.ReLU(),
        nn.Linear(128,64),
        nn.ReLU(),
        nn.Linear(64,10)
    )

  def forward(self,x):
    return self.model(x)

In [ ]:
#set learning rate and epochs
learning_rate=0.1
epochs=100

In [ ]:
#instatiate the model
model=MyNN(X_train.shape[1])

#loss function
criterion=nn.CrossEntropyLoss()

# optimizer
optimizer=optim.SGD(model.parameters(),lr=learning_rate)

In [ ]:
#training loop
for epoch in range(epochs):
  total_epoch_loss=0

  for batch_features,batch_labels in train_loader:
    #forward pass
    outputs=model(batch_features)
    #calculate loss
    loss=criterion(outputs,batch_labels)
    #backward pass
    optimizer.zero_grad()
    loss.backward()
    #update weights
    optimizer.step()
    total_epoch_loss=total_epoch_loss+loss.item()
  avg_loss=total_epoch_loss/len(train_loader)
  print(f'Epoch: {epoch+1}, Loss: {avg_loss:.4f}')

Epoch: 1, Loss: 1.3294
Epoch: 2, Loss: 0.7639
Epoch: 3, Loss: 0.6496
Epoch: 4, Loss: 0.5794
Epoch: 5, Loss: 0.5347
Epoch: 6, Loss: 0.4955
Epoch: 7, Loss: 0.4641
Epoch: 8, Loss: 0.4471
Epoch: 9, Loss: 0.4212
Epoch: 10, Loss: 0.4033
Epoch: 11, Loss: 0.3901
Epoch: 12, Loss: 0.3697
Epoch: 13, Loss: 0.3599
Epoch: 14, Loss: 0.3333
Epoch: 15, Loss: 0.3255
Epoch: 16, Loss: 0.3095
Epoch: 17, Loss: 0.3000
Epoch: 18, Loss: 0.2855
Epoch: 19, Loss: 0.2788
Epoch: 20, Loss: 0.2802
Epoch: 21, Loss: 0.2520
Epoch: 22, Loss: 0.2490
Epoch: 23, Loss: 0.2337
Epoch: 24, Loss: 0.2315
Epoch: 25, Loss: 0.2263
Epoch: 26, Loss: 0.2197
Epoch: 27, Loss: 0.2114
Epoch: 28, Loss: 0.1948
Epoch: 29, Loss: 0.1896
Epoch: 30, Loss: 0.1931
Epoch: 31, Loss: 0.1986
Epoch: 32, Loss: 0.1919
Epoch: 33, Loss: 0.1753
Epoch: 34, Loss: 0.1658
Epoch: 35, Loss: 0.1664
Epoch: 36, Loss: 0.1619
Epoch: 37, Loss: 0.1526
Epoch: 38, Loss: 0.1697
Epoch: 39, Loss: 0.1531
Epoch: 40, Loss: 0.1312
Epoch: 41, Loss: 0.1365
Epoch: 42, Loss: 0.1307
E

In [ ]:
#set model to evaluate mode
model.eval()

MyNN(
  (model): Sequential(
    (0): Linear(in_features=784, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=10, bias=True)
  )
)

In [ ]:
#evaluation code
total_correct=0
total_samples=0
with torch.no_grad():
  for batch_features,batch_labels in test_loader:
    outputs=model(batch_features)
    _,predicted=torch.max(outputs,1)
    total_samples+=batch_labels.size(0)
    total_correct+=(predicted==batch_labels).sum().item()

accuracy=total_correct/total_samples
print(f'Accuracy: {accuracy*100:.2f}%')

Accuracy: 82.75%
